#### Setup

This notebook is intended to run within Snowflake. In the sql query below, update the database name to the one you created when you accepted the Coridal Secure Data Share.

In [ ]:
-- choose the database for the Coridal Secure Data Share
use database messaging_data_personalized_user_profiles__events;

#### Contacts per Channel

In this query we will determine the number of contacts for each channel. The defined channels for each contact are stored in a object keyed by the channel name. We will use the `FLATTEN()` function to query the channel object.

In [ ]:
SELECT
  f.key as channels,
  count(*) as number_contacts
FROM
  public.contacts c
JOIN
  lateral flatten(input => c.channels) f
GROUP BY
  f.key
ORDER BY
  number_contacts DESC

#### Daily Message Stats

In this query we will determine the high-level message stats per day, including sends, opens, clicks, and bounces. We will use the `PIVOT()` function to display each stat as its own column.

In [ ]:
WITH action_totals AS (
  SELECT 
    to_date(action_date) as action_date,
    action
  FROM
    public.contactactivity
  WHERE
    action_date BETWEEN '2023-07-01' AND '2023-07-07'
    AND action IN ('message-sent', 'open', 'click', 'bounce')
)
SELECT
 *
FROM
 action_totals
PIVOT (
 count(action) FOR action IN ('message-sent', 'open', 'click', 'bounce')
) as p
ORDER BY
  action_date

#### Daily Orders Stats

In this query we will determine a summary of orders data aggregated per day for a given date range.

In [ ]:
SELECT
  to_date(purchase_date) as purchase_date,
  count(*) as number_orders,
  count(distinct cid) as number_customers,
  round(sum(total_amount),2) as total_revenue,
  round(avg(total_amount),2) as average_order_amount,
  round(avg(array_size(items))) as average_number_items
FROM
  public.orders
WHERE
  purchase_date BETWEEN '2023-07-01' and '2023-07-07'
GROUP BY
  to_date(purchase_date)
ORDER BY
  to_date(purchase_date);

#### Emails Sent per Day

In this query we will find the total number of emails sent per day for a month. We will also include the number of unique contacts that were sent an email each day. The results of this query will be used to produce a bar chart.

In [ ]:
SELECT 
  to_date(action_date) as send_date,
  count(*) as emails_sent,
  count(DISTINCT cid) as unique_contacts
FROM
  public.contactactivity
WHERE
  send_date BETWEEN '2023-07-01' AND '2023-07-31'
  and action = 'message-sent'
  and channel_type = 'email'
GROUP BY
  send_date
ORDER BY
  send_date ASC

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
#from snowflake.snowpark.context import get_active_session
#session = get_active_session()

# Turn the `emails_sent` cell into a Pandas dataframe
my_df = emails_sent.to_pandas()

# Chart the data
st.subheader("Emails Sent per Day")
st.bar_chart(my_df, x='SEND_DATE', y='EMAILS_SENT')